# Processus complet de detection de liserés

Voici tout le processus de détection automatique des liserés contenu en un seul fichier. Il sera amélioré au fur et à mesure, mais les perofrmances dépendent surtout des réseaux de neurones utilisés, et pas vraiment du code contenu dans ce notebook.

Ce notebook utilise:

- Une grande carte (c.à.d. une image reconstruite représentant une grande surface d'échantillon à analyser)
- Un réseau de neurones

Ce notebook produit:

- Une image qui correspond à la superposition de la grande carte donnée en entrée, et des prédictions faites par le réseau de neurones sur cette carte

## Sections

1. **Paramètres**

- Un ensemble de variables à modifier pour faire configurer le processus. C'est normalement la seule partie du code que vous devrez manipuler.

2. **Imports et fonctions utilitaires**

- Importe toutes les bibliothèques nécéssaires et déclare des fonctions/classes utilitaires qui servent pour le reste du notebook

3. **Découpage de la carte**

- Permet de découper une grande carte en un jeu de données adapté au réseau de neurones. Ne doit être executé qu'une seule fois pour chaque carte.

4. **Chargement du réseau de neurones**

- Charge le réseau de neurones. Le construit automatiquement à partir de ses poids si nécéssaire.
   
5. **Prédiction sur la carte**

- Effectue l'analyse de la grande carte par le réseau de neurones

6. **Superposition et sauvegarde**

- Superpose la prédiction à la carte, et effectue la sauvegarde. Cette section peut être ré-exécutée après avoir modifié certains paramètres. 

## Comment utiliser ce notebook ?

Allez dans la section **1. Paramètres** et modifiez les paramètres appropriés, puis appuyez sur le bouton *Restart Kernel and Run all Cells* (ou quelque chose qui y ressemble, en fonction de la version de jupyter utilisée) en haut du notebook.

## Autres informations

- En python, pour commenter une ligne, on utilise le symbole *#* (équivalent au *%* de matlab)
- Pour exécuter une seule cellule, selectionnez la cellule et utilisez le raccourci **Majuscule+Entrée**. Cela fonctionne aussi avec les cellules en *markdown* comme celle qui contient ce texte.


# 1. Paramètres

## 1.1 Paramètres de la grande carte et du dataset

***
**Création du dataset**

L'opération de découpage de la grande carte en dataset n'a besoin d'être effectuée qu'une seule fois. Elle découpe la grande image en entrée en une collection de plus petites images (le *dataset*), et les enregistre sur votre ordinateur.

Pour ne pas effectuer cette opération (car le dataset a déjà été généré):
> creation_dataset = False

Pour effectuer cette opération (première fois que vous analysez cette grande carte):
> creation_dataset = True

In [ ]:
creation_dataset = True

***
**Chemin vers le dossier qui contient le dataset.**

- Si *creation_dataset = True*, c'est le chemin dans lequel sera enregistré le dataset.
- Ne pas oublier de mettre un '/' à la fin
- Le dataset contiendra un dossier /images/ et un fichier dimensions.csv

In [ ]:
chemin_dataset = "/home/jbgodefroy/Documents/Data/Echantillons_A_et_D/test_algo/"

***
**Chemin vers l'image contenant la grande carte**. 

L'extension n'importe pas (jpeg, png, tif...).  Si l'image est en couleur, elle sera convertie automatiquement en noir et blanc.

In [ ]:
chemin_grande_carte = "/home/jbgodefroy/Documents/Data/Echantillons_A_et_D/Serie_A/ech28_reconstruction_x5_grayscale.png"

***

## 1.2 Paramètres de l'analyse

***
**Chemin et nom de fichier où enregistrer l'image de prédiction**. L'extension n'importe pas (jpeg, png, tif...).

In [ ]:
chemin_prediction = "/home/jbgodefroy/Documents/Data/Echantillons_A_et_D/test_algo/prediction.png"

***
**Chemin où trouver le réseau de neurones**. Le chemin peut pointer vers:
- un fichier de réseau de neurones (extension *.keras*),
- un dossier contenant un réseau de neurone (extension *.keras*),
- un dossier contenant un fichier *model_info.json* et un sous-dossier *weights*. Un fichier *model.keras* sera alors automatiquement généré à partir du dossier *weights*.

In [ ]:
chemin_reseau_neurones = "./trained_models/2025-02-07_Resnet_64x2x11_64x2x7_128x2x5/model_500_epochs.keras"

***
**Seuil de prédiction et paramètres d'affichage**

Après l'analyse, la prédiction est superposée à l'image. Les valeurs de prédiction vont de 0 à 255, le paramètre *seuil_prediction* permet d'afficher uniquement les zones dépassant le seuil.

La meilleur valeur de *seuil_prediction* varie en fonction du réseau de neurones utilisé, je recommande de commencer à *seuil_prediction = 128* et d'ajuster après l'analyse.

In [ ]:
seuil_prediction = 0

# Parfois, l'échelle des valeurs de prédiction doit être inversée, cela dépend du réseau de neurones
inverser_prediction = False

# Pour afficher la prédiction sur toutes la grande carte, et pas uniquement dans les zones classée comme contenant un liseré
ignorer_classification = False

# 2. Imports et utilitaires

In [ ]:
import os, shutil
import datetime
from functools import partial

import numpy as np
import pandas as pd
import cv2

import tensorflow as tf
from tensorflow.keras import layers

import keras
from keras.models import Sequential
from keras.layers import *

import matplotlib.pyplot as plt
import pathlib
import json

In [ ]:
size = (400, 400)  # Specify the size of the sub-images (width x height)
overlap = (133, 133)  # Specify the overlap between sub-images (horizontal x vertical)
use_mask = False # True to use an annotation mask
make_annotations = False
side_crop = 100 # Big map side crop in pixels

In [ ]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=5, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, kernel_size=5, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.filters = filters
        self.strides = strides
        self.kernel_size = kernel_size
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters, kernel_size=kernel_size),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "activation": self.activation,
            "filters": self.filters,
            "strides": self.strides,
            "kernel_size": self.kernel_size
        })
        return config   
    

class SEBlock(tf.keras.layers.Layer):
    def __init__(self, ratio=16, **kwargs):
        super().__init__(**kwargs)
        self.ratio = ratio

    def build(self, input_shape):
        num_channels = input_shape[-1]
        self.squeeze = tf.keras.layers.GlobalAveragePooling2D()
        self.dense_1 = tf.keras.layers.Dense(num_channels // self.ratio, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_channels, activation='sigmoid')

    def call(self, inputs):
        x = self.squeeze(inputs)
        x = self.dense_1(x)
        x = self.dense_2(x)
        x = tf.keras.layers.Reshape((1, 1, -1))(x)
        return inputs * x

    def get_config(self):
        config = super().get_config()
        config.update({"ratio": self.ratio})
        return config

class SE_ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", se_ratio=16, **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.filters = filters
        self.strides = strides
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]
        self.se = SEBlock(ratio=se_ratio)

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        Z = self.se(Z)  # Appliquer le bloc SE ici.
        return self.activation(Z + skip_Z)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "activation": self.activation,
            "filters": self.filters,
            "strides": self.strides,
            "se_ratio": self.se.ratio
        })
        return config

class adaptateur(tf.keras.Model):

    def __init__(self, n=1):
        super(adaptateur, self).__init__()

        self.gap = GlobalAveragePooling2D()
        self.dense = Dense(n, activation='sigmoid', bias_initializer=None, use_bias=False)

    def get_weights(self, n):
        return np.reshape(self.dense.weights[0].numpy(), n)

        
    def call(self, x):

        x = self.gap(x)
        x = self.dense(x)

        return x

In [ ]:
def get_resnet2(filters_sequence = [[64, 3, 5], [128, 4, 5], [256, 6, 5], [512, 3, 5]], input_shape=[400, 400, 1], first_kernel_size=7, se_blocks=False, activation='relu'):

    resnet = tf.keras.Sequential([
        DefaultConv2D(64, kernel_size=first_kernel_size, strides=2, input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=1, padding="same"),
    ])

    filters_list = [e for sublist in [[(f, kernel_size)]*n for f, n, kernel_size in filters_sequence] for e in sublist]
    prev_filters = filters_list[0]

    for filters, kernel_size in filters_list:
        strides = 1 if filters == prev_filters else 2
        if se_blocks:
            resnet.add(SE_ResidualUnit(filters, strides=strides, activation=activation))
        else:
            resnet.add(ResidualUnit(filters, strides=strides, kernel_size=kernel_size))
        prev_filters = filters

    return resnet


# 3. Découpage de la carte

In [ ]:
size = (400, 400)  # Specify the size of the sub-images (width x height)
image_size = size[0]
overlap = (133, 133)  # Specify the overlap between sub-images (horizontal x vertical)
use_mask = False # True to use an annotation mask
make_annotations = False
side_crop = 100 # Big map side crop in pixels

In [ ]:
def split_image(image, out_path, size, overlap):
    if len(image.shape) == 3:
        height, width, _ = image.shape
    else:
        height, width = image.shape
    size_x, size_y = size
    overlap_x, overlap_y = overlap

    # Calculate the step size to spli t the image with overlap
    step_x = size_x - overlap_x
    step_y = size_y - overlap_y
    i = 0
    dx, dy = 0, 0
    for y in range(0, height - size_y + 1, step_y):
        dy+=1
        for x in range(0, width - size_x + 1, step_x):
            if dy == 1:
                dx+=1
            # Extract the sub-image
            sub_image = image[y:y+size_y, x:x+size_x]

            # Save the sub-image
            filename = f"{out_path}image_{i}_x{x}_y{y}.png"

            sub_image = cv2.normalize(sub_image, None, 0, 255, cv2.NORM_MINMAX, dtype=0)
            cv2.imwrite(filename, sub_image)
            i += 1
            
    return dx, dy

In [ ]:
image = cv2.imread(chemin_grande_carte, cv2.IMREAD_GRAYSCALE)[side_crop:-side_crop, side_crop:-side_crop]

In [ ]:
if creation_dataset:
    images_path = chemin_dataset + 'images/'
    
    if os.path.isdir(images_path):
        for e in os.listdir(images_path):
            os.unlink(images_path + e)
    else:
        os.mkdir(images_path)

    columns, rows = split_image(image, images_path, size, overlap)
    pd.DataFrame({'rows':rows, 'columns':columns}, index=[0]).to_csv(chemin_dataset + 'dimensions.csv')
else:
    rows, columns = pd.read_csv(chemin_dataset + 'dimensions.csv', index_col=0).iloc[0]

# 4. Chargement du réseau de neurones

In [ ]:
custom_objects={'ResidualUnit':ResidualUnit,
                'SE_ResidualUnit':SE_ResidualUnit,
                'adaptateur':adaptateur}

In [ ]:
chemin_reseau_neurones = pathlib.Path(chemin_reseau_neurones)

if chemin_reseau_neurones.suffix == '.keras':
    model = tf.keras.models.load_model(chemin_reseau_neurones, custom_objects=custom_objects)
    if len(model.layers) == 3:
        segnet, decoder = model.layers[1:]
    else:
        segnet, decoder = model.layers
    
elif chemin_reseau_neurones.is_dir():
    found = False
    for e  in sorted(os.listdir(chemin_reseau_neurones)):
        if e.endswith('.keras'):
            model = tf.keras.models.load_model(chemin_reseau_neurones.joinpath(e), custom_objects=custom_objects)
            if len(model.layers) == 3:
                segnet, decoder = model.layers[1:]
            else:
                segnet, decoder = model.layers
            found = True
            break        
    if not found and chemin_reseau_neurones.joinpath('weights').is_dir() and chemin_reseau_neurones.joinpath('model_info.json').is_file():
        with open(chemin_reseau_neurones.joinpath('model_info.json'), 'r') as f:
            JSON = json.load(f)
        segnet = globals()[JSON['generator']](**JSON['generator_parameters'])
        decoder = adaptateur()
        model = Sequential([segnet, decoder])
        del JSON

        weights = []
        for i in range(len(model.get_weights())):
            weights.append(np.load(chemin_reseau_neurones.joinpath('weights').joinpath(f'weights_{i}.npy')))
        model.set_weights(weights)
        
        tf.keras.models.save_model(model, chemin_reseau_neurones.joinpath('model.keras'))
        


# 5. Prédiction sur la carte

In [ ]:
custom_objects={'ResidualUnit':ResidualUnit,
                'SE_ResidualUnit':SE_ResidualUnit,
                'adaptateur':adaptateur}


weights = decoder.get_weights(n=decoder.layers[1].weights[0].shape[0])

image_size         = model.input.shape[1:3][0]
segnet_output_size = segnet.output.shape[1:3][0]
delta = int(segnet_output_size - segnet_output_size*overlap[0]/image_size)

In [ ]:
def sortbynum(s : str):
    split = s.split('_')
    return int(split[1])

images = np.empty((len(os.listdir(chemin_dataset+'images/')), image_size, image_size))
for i, f in enumerate(sorted(os.listdir(chemin_dataset+'images/'), key=sortbynum)):
    images[i] = cv2.imread(chemin_dataset+'images/'+f, cv2.IMREAD_GRAYSCALE)

### Calcul de l'image moyenne (pour filtrer les effets indésirables sur les bords des images)

In [ ]:
thresh = 0.5

avg = np.zeros((segnet_output_size, segnet_output_size))
global_min = None
for j in range(rows):
    vis_images = images[columns*(j):columns*(j+1)]

    prediction_raw = segnet(vis_images)
    prediction = np.average(prediction_raw, weights=weights,axis=3)
    
    if global_min:
        global_min = np.min([global_min, np.min(prediction)])
    else:
        global_min = np.min(prediction)
    
    for i in range(columns):
        avg += prediction[i]

avg /= (rows*columns)

### Prédictions

In [ ]:
stitch = np.empty((rows*delta, columns*delta))

empty_patch = np.ones((segnet_output_size, segnet_output_size))*global_min

for j in range(rows):
    vis_images = images[columns*(j):columns*(j+1)]

    prediction_raw = segnet(vis_images)
    prediction = np.average(prediction_raw, weights=weights,axis=3)
    
    predicted_labels = decoder(prediction_raw)
    for i in range(columns):
        
        
        if (not ignorer_classification) and predicted_labels[i] < thresh:
            patch = empty_patch
        else:
            patch = (prediction[i]-avg) * (-1 if inverser_prediction else 1)
            
        stitch[j*delta:(j+1)*delta:, i*delta:(i+1)*delta] = patch[segnet_output_size-delta:, segnet_output_size-delta:]
        #stitch[j*delta:(j+1)*delta:, i*delta:(i+1)*delta] = cv2.normalize(stitch[j*delta:(j+1)*delta:, i*delta:(i+1)*delta], None, 0, 255, cv2.NORM_MINMAX, dtype=0)

stitch = np.maximum(stitch, np.min(stitch[stitch != global_min]))
stitch = cv2.normalize(stitch, None, 0, 255, cv2.NORM_MINMAX, dtype=0)

### Nettoyage

In [ ]:
del model, segnet, decoder
del prediction_raw, prediction, predicted_labels
del images

# 6. Superposition et sauvegarde

Après avoir modifé le paramètre *seuil_prediction*, il n'est pas nécéssaire de refaire toute l'analyse pour générer la superposition avec des seuils différents:

1. Modifier la valeur de *seuil_prediction* et éxécuter la case contenant *seuil_prediction* avec  **Maj**+**Entrée**
2. Cliquer sur ce texte
3. Appuyer sur **Maj**+**Entrée** (la case de code en dessous devrait maintenant s'afficher et être séléctionnée automatiquement)
4. Ré-appuyer sur **Maj**+**Entrée** pour éxécuter la case ci dessous (la date est l'heure devraient s'afficher sous la case, pour confirmer que la nouvelle sauvegarde a été effectuée)

Pour masquer à nouveau le code, vous pouvez cliquer sur le petit triangle à côté du **5**, en haut à gauche de cette case.

In [ ]:
new_sz = (133 + 266 * columns, 133 + 266 * rows)
resized_stitch = cv2.resize(stitch, new_sz)
resized_stitch = np.clip(resized_stitch, a_min=seuil_prediction, a_max=256)
resized_stitch = cv2.normalize(resized_stitch, 0, 256,  norm_type=cv2.NORM_MINMAX, dtype=8)
cropped_image = image[:new_sz[1], :new_sz[0]]

# Ne pas tout faire d'un coup sinon le noyau crash
blend = np.empty((new_sz[1], new_sz[0], 3), dtype=int)
# CV2 enregistre en BGR
blend[..., 0] = cropped_image
blend[..., 1] = cropped_image
blend[..., 2] = cv2.addWeighted(cropped_image, 1, resized_stitch, 1.5, 1)

cv2.imwrite(chemin_prediction, blend)
print("Image sauvegardée: ", datetime.datetime.now())

# Fin !

In [ ]:
print("Fin de l'éxécution: ", datetime.datetime.now())
plt.imshow(blend)